In this project I want to run a regression comparing the effectiveness of US aid in global terms, across implementing agencies (USAID, State Department, Dept. of the Interior, etc.) and the effect it has on Human Development Index per country. I do this by accesing a database from explorer.usaid.gov and population data from the UN and adapting it to my needs (cleaning, rea-aligning, etc.). 

Second, I want to run a regression comparing aid implemented by USAID per sector and compare it's effectiveness compared to Agriculture aid. Again, I used a database from explorer.usaid.gov and adapted it to my needs. 

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
df_HDI=pd.read_csv(r'HDI.csv')

In [3]:
df_HDI.head()

,HDI Rank,Country,1990,Unnamed: 3,1991,Unnamed: 5,1992,Unnamed: 7,1993,Unnamed: 9,...,2014,Unnamed: 51,2015,Unnamed: 53,2016,Unnamed: 55,2017,Unnamed: 57,2018,Unnamed: 59
0,Human Development Index (HDI)\nnull\nDimension...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,170,Afghanistan,0.298,NaN,0.304,NaN,0.312,NaN,0.308,NaN,...,0.488,NaN,0.490,NaN,0.491,NaN,0.493,NaN,0.496,NaN
2,69,Albania,0.644,NaN,0.625,NaN,0.608,NaN,0.611,NaN,...,0.787,NaN,0.788,NaN,0.788,NaN,0.789,NaN,0.791,NaN
3,82,Algeria,0.578,NaN,0.582,NaN,0.589,NaN,0.593,NaN,...,0.749,NaN,0.751,NaN,0.755,NaN,0.758,NaN,0.759,NaN
4,36,Andorra,..,NaN,..,NaN,..,NaN,..,NaN,...,0.853,NaN,0.850,NaN,0.854,NaN,0.852,NaN,0.857,NaN


In [4]:
#get rid of white spaces in the country column strings

df_HDI['Country']=df_HDI['Country'].str.strip()

In [5]:
#delete unnamed columns from df_HDI

list_unwanted_HDI=[]
[list_unwanted_HDI.append(x) for x in df_HDI.columns if 'Unnamed:' in x];
[df_HDI.drop(x, axis=1, inplace=True) for x in list_unwanted_HDI]

df_HDI.head(5)


,HDI Rank,Country,1990,1991,1992,1993,1994,1995,1996,1997,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Human Development Index (HDI)\nnull\nDimension...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,170,Afghanistan,0.298,0.304,0.312,0.308,0.303,0.327,0.331,0.335,...,0.447,0.464,0.465,0.479,0.485,0.488,0.490,0.491,0.493,0.496
2,69,Albania,0.644,0.625,0.608,0.611,0.617,0.629,0.639,0.639,...,0.729,0.740,0.759,0.771,0.781,0.787,0.788,0.788,0.789,0.791
3,82,Algeria,0.578,0.582,0.589,0.593,0.597,0.602,0.610,0.619,...,0.720,0.730,0.738,0.737,0.746,0.749,0.751,0.755,0.758,0.759
4,36,Andorra,..,..,..,..,..,..,..,..,...,0.830,0.828,0.827,0.849,0.846,0.853,0.850,0.854,0.852,0.857


In [6]:
#get rid of the HDI Rank column, and the first row.

df_HDI=df_HDI.iloc[1:,1:]

#replace those '..' (no data) and drop all nan from rows (countries)

df_HDI.replace({'..':np.nan},inplace=True)
df_HDI.dropna(axis=0, how='any',inplace=True)
df_HDI.head()


,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
1,Afghanistan,0.298,0.304,0.312,0.308,0.303,0.327,0.331,0.335,0.339,...,0.447,0.464,0.465,0.479,0.485,0.488,0.490,0.491,0.493,0.496
2,Albania,0.644,0.625,0.608,0.611,0.617,0.629,0.639,0.639,0.649,...,0.729,0.740,0.759,0.771,0.781,0.787,0.788,0.788,0.789,0.791
3,Algeria,0.578,0.582,0.589,0.593,0.597,0.602,0.610,0.619,0.629,...,0.720,0.730,0.738,0.737,0.746,0.749,0.751,0.755,0.758,0.759
7,Argentina,0.707,0.714,0.719,0.725,0.729,0.731,0.738,0.746,0.752,...,0.810,0.818,0.823,0.823,0.824,0.825,0.828,0.828,0.832,0.830
8,Armenia,0.633,0.629,0.585,0.590,0.600,0.604,0.614,0.625,0.637,...,0.724,0.729,0.733,0.737,0.743,0.746,0.748,0.751,0.758,0.760


In [7]:
#check if there are nulls left in the rows(countries)

df_HDI.isnull().any().any()

False

In [8]:
#read in aid(gross) per country, across all US implementing agencies 

df_FDAC=pd.read_csv(r'us_foreign_aid_country.csv')

In [9]:
df_FDAC.head(5)

,country_code,country_name,region_id,region_name,income_group_acronym,income_group_name,transaction_type_id,transaction_type_name,fiscal_year,current_amount,constant_amount
0,ABW,Aruba,6,Western Hemisphere,HIC,High Income Country,2,Obligations,1999,19000,27957
1,ABW,Aruba,6,Western Hemisphere,HIC,High Income Country,2,Obligations,2000,50000,72090
2,ABW,Aruba,6,Western Hemisphere,HIC,High Income Country,2,Obligations,2004,1000,1330
3,ABW,Aruba,6,Western Hemisphere,HIC,High Income Country,2,Obligations,2005,29270,37766
4,ABW,Aruba,6,Western Hemisphere,HIC,High Income Country,2,Obligations,2006,1000,1250


In [10]:
#filter for actual aid paid out

df_FDAC=df_FDAC[df_FDAC['transaction_type_name']=='Disbursements']

In [11]:
#check for relevant years (I will have to adapt the other dataframes later)

df_FDAC['fiscal_year'].unique()

array(['2004', '2005', '2006', '2007', '2008', '2009', '2001', '2002',
       '2003', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020'], dtype=object)

In [12]:
#let's interate for a second and filter for only these years in the HDI df

new_columns_hdi=['2004', '2005', '2006', '2007', '2008', '2009', '2001', '2002',
       '2003', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020']

old_columns_hdi=['1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018']

[df_HDI.drop(column, axis=1, inplace=True) for column in old_columns_hdi if column not in new_columns_hdi];

df_HDI.head()

,Country,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
1,Afghanistan,0.347,0.378,0.387,0.400,0.410,0.419,0.431,0.436,0.447,0.464,0.465,0.479,0.485,0.488,0.490,0.491,0.493,0.496
2,Albania,0.673,0.680,0.687,0.692,0.702,0.709,0.718,0.724,0.729,0.740,0.759,0.771,0.781,0.787,0.788,0.788,0.789,0.791
3,Algeria,0.655,0.666,0.676,0.685,0.694,0.699,0.708,0.711,0.720,0.730,0.738,0.737,0.746,0.749,0.751,0.755,0.758,0.759
7,Argentina,0.775,0.770,0.775,0.775,0.777,0.801,0.806,0.810,0.810,0.818,0.823,0.823,0.824,0.825,0.828,0.828,0.832,0.830
8,Armenia,0.653,0.663,0.672,0.681,0.694,0.708,0.724,0.727,0.724,0.729,0.733,0.737,0.743,0.746,0.748,0.751,0.758,0.760


In [13]:
#reset the index, drop the old one (numbers are all crazy because of the disbursement's filter)

df_FDAC.reset_index(drop=True, inplace=True)

In [14]:
#We only really need the country names, years paid, amount.. we can filter all other columns

df_FDAC=df_FDAC[['country_name','fiscal_year','constant_amount']]

In [15]:
df_FDAC.head(5)

,country_name,fiscal_year,constant_amount
0,Aruba,2004,1330
1,Aruba,2005,37766
2,Aruba,2006,1250
3,Aruba,2007,74944
4,Aruba,2008,13281


In [16]:
# We are goint to need this df with years as the column headers...

df_FDAC_CA= pd.pivot_table(df_FDAC, values='constant_amount', index=['country_name'],

                    columns=['fiscal_year'], aggfunc=np.sum)

#lets check for Nans by countries any get rid of these countries
print('number of nans by country:',df_FDAC_CA.T.isnull().any().sum())

df_FDAC_CA.dropna(axis=0,how='any', inplace=True)

number of nans by country: 83


In [17]:
#lets reset the index

df_FDAC_CA.reset_index(drop=False, inplace=True)

In [18]:
#rename the country column so that both dataframes have the same name

df_FDAC_CA.rename(columns={'country_name':'Country'},inplace=True)

In [19]:
#lets take a look....

df_FDAC_CA

fiscal_year,Country,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Afghanistan,5.745670e+07,5.549478e+08,9.576053e+08,1.648764e+09,2.143609e+09,4.197466e+09,5.964489e+09,9.938714e+09,1.009106e+10,...,1.311957e+10,1.136811e+10,1.100192e+10,7.576634e+09,9.944102e+09,4.589219e+09,5.242978e+09,5.281673e+09,4.669036e+09,7.972394e+08
1,Albania,5.876006e+07,7.933978e+07,7.183047e+07,8.319374e+07,6.165720e+07,6.080430e+07,5.100935e+07,4.767786e+07,4.540912e+07,...,3.836475e+07,3.107365e+07,3.390559e+07,2.754776e+07,2.478180e+07,3.047301e+07,3.821392e+07,2.240495e+07,2.481227e+07,7.826191e+06
2,Algeria,5.746860e+05,4.616154e+06,5.031511e+06,5.319066e+06,3.157536e+06,1.938132e+06,3.624181e+06,1.169191e+07,1.067139e+07,...,1.204787e+07,1.412232e+07,1.358960e+07,1.002147e+07,9.636712e+06,8.632287e+06,2.008797e+07,6.161996e+06,1.796278e+07,9.702680e+05
3,Angola,9.071448e+07,1.539027e+08,2.030413e+08,1.716597e+08,1.086911e+08,7.709941e+07,5.273730e+07,6.630510e+07,5.746754e+07,...,7.797982e+07,9.103081e+07,8.456166e+07,7.324076e+07,6.663251e+07,7.341603e+07,5.941320e+07,5.314818e+07,5.126563e+07,1.857171e+07
4,Antigua and Barbuda,1.121219e+06,5.057372e+06,4.520190e+05,2.036993e+06,8.489620e+05,5.588330e+05,6.572340e+05,1.264000e+05,8.311740e+05,...,4.557690e+05,5.448690e+05,5.803000e+05,5.547160e+05,4.123540e+05,6.749740e+05,8.098460e+05,3.790000e+03,1.349560e+05,4.796900e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,West Bank/Gaza,9.084436e+07,1.820396e+08,2.568901e+08,2.256842e+08,2.679466e+08,2.101506e+08,2.455159e+08,4.210612e+08,1.082774e+09,...,9.528217e+08,3.716525e+08,8.872935e+08,4.197906e+08,4.892322e+08,6.542347e+08,3.919172e+08,2.737646e+08,1.200856e+08,1.315109e+07
160,World,3.839000e+09,4.809014e+09,3.800297e+09,7.517570e+09,7.052233e+09,5.438266e+09,5.950804e+09,7.938419e+09,8.427445e+09,...,8.803727e+09,1.071732e+10,1.076323e+10,1.026097e+10,1.004176e+10,1.188456e+10,1.111592e+10,1.135966e+10,1.139739e+10,5.411792e+09
161,Yemen,3.560892e+06,3.048766e+07,4.625275e+07,7.684400e+07,3.894842e+07,5.222387e+07,4.201206e+07,3.628785e+07,3.431733e+07,...,1.137508e+08,2.237021e+08,2.912659e+08,2.620217e+08,1.690273e+08,2.893510e+08,4.561710e+08,5.763233e+08,6.930979e+08,5.013316e+08
162,Zambia,4.406611e+07,7.104028e+07,7.684341e+07,8.432438e+07,1.166802e+08,1.513492e+08,2.089135e+08,2.477826e+08,2.665279e+08,...,2.861896e+08,3.349827e+08,3.662650e+08,3.666934e+08,3.076268e+08,3.694260e+08,4.965429e+08,4.826898e+08,3.846704e+08,1.591572e+08


In [20]:
#let's make a standard country codes column in both our dfs for mergin purposes

#first HDI

from hdx.location.country import Country
Country.countriesdata(use_live=False, country_name_overrides={'PSE': 'oPt'})

iso_countries_hdi = []


for x in df_HDI['Country']:
    try:
        iso3 = Country.get_iso3_country_code_fuzzy(x)
        info = Country.get_country_info_from_iso3(iso3[0])
        reg=info.get('#country+code+v_iso3')
#        reg = info.get('#country+name+preferred')
        iso_countries_hdi.append(reg)
        
    except AttributeError:
        iso_countries_hdi.append(x)

df_HDI['iso3_codes']=iso_countries_hdi

#second aid


iso_countries_FDAC_CA = []


for x in df_FDAC_CA['Country']:
    try:
        iso3 = Country.get_iso3_country_code_fuzzy(x)
        info = Country.get_country_info_from_iso3(iso3[0])
        reg=info.get('#country+code+v_iso3')
#        reg = info.get('#country+name+preferred')
        iso_countries_FDAC_CA.append(reg)
        
    except AttributeError:
        iso_countries_FDAC_CA.append(x)

df_FDAC_CA['iso3_codes']=iso_countries_FDAC_CA

C:\Users\dccam\anaconda3\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


0


In [21]:
#let's merge both dataframes

merged_hdi_aidg=df_HDI.merge(df_FDAC_CA, on='iso3_codes')

#it seems not all years are represented by each dataframe
merged_hdi_aidg.columns

Index(['Country_x', '2001_x', '2002_x', '2003_x', '2004_x', '2005_x', '2006_x',
       '2007_x', '2008_x', '2009_x', '2010_x', '2011_x', '2012_x', '2013_x',
       '2014_x', '2015_x', '2016_x', '2017_x', '2018_x', 'iso3_codes',
       'Country_y', '2001_y', '2002_y', '2003_y', '2004_y', '2005_y', '2006_y',
       '2007_y', '2008_y', '2009_y', '2010_y', '2011_y', '2012_y', '2013_y',
       '2014_y', '2015_y', '2016_y', '2017_y', '2018_y', '2019', '2020'],
      dtype='object')

In [22]:
#lets leave out any year that is not present in both dataframes

merged_hdi_aidg=merged_hdi_aidg.loc[:, merged_hdi_aidg.columns.str.contains('|'.join(['_y', '_x','iso3']))]

In [23]:
#this is our final country by years dataframe 

merged_hdi_aidg.columns=['Country', '2001_x', '2002_x', '2003_x', '2004_x', '2005_x', '2006_x',
       '2007_x', '2008_x', '2009_x', '2010_x', '2011_x', '2012_x', '2013_x',
       '2014_x', '2015_x', '2016_x', '2017_x', '2018_x', 'iso3_codes',
       'Country_drop', '2001_y', '2002_y', '2003_y', '2004_y', '2005_y', '2006_y',
       '2007_y', '2008_y', '2009_y', '2010_y', '2011_y', '2012_y', '2013_y',
       '2014_y', '2015_y', '2016_y', '2017_y', '2018_y']

merged_hdi_aidg.drop('Country_drop',axis=1,inplace=True)

In [24]:
merged_hdi_aidg

,Country,2001_x,2002_x,2003_x,2004_x,2005_x,2006_x,2007_x,2008_x,2009_x,...,2009_y,2010_y,2011_y,2012_y,2013_y,2014_y,2015_y,2016_y,2017_y,2018_y
0,Afghanistan,0.347,0.378,0.387,0.400,0.410,0.419,0.431,0.436,0.447,...,1.009106e+10,9.165498e+09,1.311957e+10,1.136811e+10,1.100192e+10,7.576634e+09,9.944102e+09,4.589219e+09,5.242978e+09,5.281673e+09
1,Albania,0.673,0.680,0.687,0.692,0.702,0.709,0.718,0.724,0.729,...,4.540912e+07,4.349609e+07,3.836475e+07,3.107365e+07,3.390559e+07,2.754776e+07,2.478180e+07,3.047301e+07,3.821392e+07,2.240495e+07
2,Algeria,0.655,0.666,0.676,0.685,0.694,0.699,0.708,0.711,0.720,...,1.067139e+07,1.160851e+07,1.204787e+07,1.412232e+07,1.358960e+07,1.002147e+07,9.636712e+06,8.632287e+06,2.008797e+07,6.161996e+06
3,Argentina,0.775,0.770,0.775,0.775,0.777,0.801,0.806,0.810,0.810,...,4.517294e+06,6.333383e+06,4.469764e+06,1.132358e+07,4.034653e+06,2.486749e+06,1.851473e+06,6.539909e+06,2.689528e+06,5.546173e+06
4,Armenia,0.653,0.663,0.672,0.681,0.694,0.708,0.724,0.727,0.724,...,1.046962e+08,1.161102e+08,1.507209e+08,9.460744e+07,7.068238e+07,4.907025e+07,3.818708e+07,4.814792e+07,3.399343e+07,3.888780e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Viet Nam,0.586,0.594,0.603,0.612,0.616,0.624,0.632,0.639,0.650,...,9.719668e+07,1.188879e+08,1.160619e+08,1.294274e+08,1.448944e+08,1.401362e+08,1.184707e+08,1.459550e+08,1.707319e+08,1.636159e+08
104,Yemen,0.449,0.456,0.464,0.471,0.472,0.478,0.486,0.488,0.503,...,3.431733e+07,6.684586e+07,1.137508e+08,2.237021e+08,2.912659e+08,2.620217e+08,1.690273e+08,2.893510e+08,4.561710e+08,5.763233e+08
105,Zambia,0.436,0.445,0.455,0.464,0.475,0.486,0.492,0.507,0.521,...,2.665279e+08,2.817670e+08,2.861896e+08,3.349827e+08,3.662650e+08,3.666934e+08,3.076268e+08,3.694260e+08,4.965429e+08,4.826898e+08
106,Zimbabwe,0.453,0.444,0.430,0.427,0.425,0.429,0.434,0.432,0.448,...,3.091313e+08,1.940942e+08,2.168274e+08,1.944087e+08,1.947890e+08,2.002338e+08,1.781875e+08,2.363469e+08,2.362431e+08,2.312752e+08


In [25]:
#lets prepare this for a clean database

old_columns=merged_hdi_aidg.columns

new_columns=[]
empty_list=[]

for item in old_columns: 
    
    if item.endswith('_x'):
        new_columns.append('X'+item[:-2])

    elif item.endswith('_y'):
        new_columns.append('Y'+item[:-2])
        
    else:
        new_columns.append(item)
        

for thing in merged_hdi_aidg['Country']:
    empty_list.append('see')
    
    
merged_hdi_aidg.columns=new_columns
merged_hdi_aidg

,Country,X2001,X2002,X2003,X2004,X2005,X2006,X2007,X2008,X2009,...,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018
0,Afghanistan,0.347,0.378,0.387,0.400,0.410,0.419,0.431,0.436,0.447,...,1.009106e+10,9.165498e+09,1.311957e+10,1.136811e+10,1.100192e+10,7.576634e+09,9.944102e+09,4.589219e+09,5.242978e+09,5.281673e+09
1,Albania,0.673,0.680,0.687,0.692,0.702,0.709,0.718,0.724,0.729,...,4.540912e+07,4.349609e+07,3.836475e+07,3.107365e+07,3.390559e+07,2.754776e+07,2.478180e+07,3.047301e+07,3.821392e+07,2.240495e+07
2,Algeria,0.655,0.666,0.676,0.685,0.694,0.699,0.708,0.711,0.720,...,1.067139e+07,1.160851e+07,1.204787e+07,1.412232e+07,1.358960e+07,1.002147e+07,9.636712e+06,8.632287e+06,2.008797e+07,6.161996e+06
3,Argentina,0.775,0.770,0.775,0.775,0.777,0.801,0.806,0.810,0.810,...,4.517294e+06,6.333383e+06,4.469764e+06,1.132358e+07,4.034653e+06,2.486749e+06,1.851473e+06,6.539909e+06,2.689528e+06,5.546173e+06
4,Armenia,0.653,0.663,0.672,0.681,0.694,0.708,0.724,0.727,0.724,...,1.046962e+08,1.161102e+08,1.507209e+08,9.460744e+07,7.068238e+07,4.907025e+07,3.818708e+07,4.814792e+07,3.399343e+07,3.888780e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Viet Nam,0.586,0.594,0.603,0.612,0.616,0.624,0.632,0.639,0.650,...,9.719668e+07,1.188879e+08,1.160619e+08,1.294274e+08,1.448944e+08,1.401362e+08,1.184707e+08,1.459550e+08,1.707319e+08,1.636159e+08
104,Yemen,0.449,0.456,0.464,0.471,0.472,0.478,0.486,0.488,0.503,...,3.431733e+07,6.684586e+07,1.137508e+08,2.237021e+08,2.912659e+08,2.620217e+08,1.690273e+08,2.893510e+08,4.561710e+08,5.763233e+08
105,Zambia,0.436,0.445,0.455,0.464,0.475,0.486,0.492,0.507,0.521,...,2.665279e+08,2.817670e+08,2.861896e+08,3.349827e+08,3.662650e+08,3.666934e+08,3.076268e+08,3.694260e+08,4.965429e+08,4.826898e+08
106,Zimbabwe,0.453,0.444,0.430,0.427,0.425,0.429,0.434,0.432,0.448,...,3.091313e+08,1.940942e+08,2.168274e+08,1.944087e+08,1.947890e+08,2.002338e+08,1.781875e+08,2.363469e+08,2.362431e+08,2.312752e+08


In [26]:
#this is the last df, population per year

df_pop=pd.read_csv(r'pop_UN.csv')

In [27]:
df_pop.head(10)


,Index,"Region, subregion, country or area *",Type,1950,1951,1952,1953,1954,1955,1956,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,1,WORLD,World,2 536 431,2 584 034,2 630 862,2 677 609,2 724 847,2 773 020,2 822 443,...,7 041 194,7 125 828,7 210 582,7 295 291,7 379 797,7 464 022,7 547 859,7 631 091,7 713 468,7 794 799
1,2,UN development groups,Label/Separator,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,3,More developed regions,Development Group,814 819,824 004,833 720,843 788,854 060,864 430,874 824,...,1 239 557,1 244 115,1 248 454,1 252 615,1 256 622,1 260 479,1 264 146,1 267 559,1 270 630,1 273 304
3,4,Less developed regions,Development Group,1 721 612,1 760 031,1 797 142,1 833 822,1 870 786,1 908 590,1 947 620,...,5 801 637,5 881 713,5 962 129,6 042 676,6 123 175,6 203 543,6 283 713,6 363 532,6 442 838,6 521 494
4,5,Least developed countries,Development Group,195 428,199 180,203 015,206 986,211 133,215 486,220 061,...,856 471,876 867,897 793,919 223,941 131,963 520,986 385,1 009 691,1 033 389,1 057 438
5,6,"Less developed regions, excluding least develo...",Development Group,1 526 184,1 560 850,1 594 126,1 626 836,1 659 653,1 693 104,1 727 558,...,4 945 165,5 004 846,5 064 335,5 123 453,5 182 043,5 240 024,5 297 327,5 353 841,5 409 449,5 464 056
6,7,"Less developed regions, excluding China",Development Group,1 157 420,1 179 933,1 203 963,1 229 440,1 256 303,1 284 497,1 313 977,...,4 394 313,4 466 549,4 539 157,4 612 005,4 684 981,4 758 019,4 831 087,4 904 155,4 977 204,5 050 208
7,8,Land-locked Developing Countries (LLDC),Special other,103 803,105 870,108 079,110 423,112 894,115 488,118 201,...,430 709,441 057,451 699,462 624,473 817,485 277,496 988,508 906,520 973,533 143
8,9,Small Island Developing States (SIDS),Special other,23 771,24 209,24 685,25 187,25 710,26 249,26 801,...,66 035,66 779,67 491,68 180,68 856,69 515,70 158,70 792,71 429,72 076
9,10,World Bank income groups,Label/Separator,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [28]:
#lets filter out what we need - just the countries

df_pop['Region, subregion, country or area *']=df_pop['Region, subregion, country or area *'].str.strip()
df_pop=df_pop[df_pop['Type']=='Country/Area']

In [29]:
df_pop

,Index,"Region, subregion, country or area *",Type,1950,1951,1952,1953,1954,1955,1956,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
26,27,Burundi,Country/Area,2 309,2 360,2 406,2 449,2 492,2 537,2 585,...,8 958,9 246,9 540,9 844,10 160,10 488,10 827,11 175,11 531,11 891
27,28,Comoros,Country/Area,159,163,167,170,173,176,179,...,707,724,742,759,777,796,814,832,851,870
28,29,Djibouti,Country/Area,62,63,65,66,68,70,71,...,854,868,883,899,914,929,944,959,974,988
29,30,Eritrea,Country/Area,822,835,849,865,882,900,919,...,3 214,3 250,3 281,3 311,3 343,3 377,3 413,3 453,3 497,3 546
30,31,Ethiopia,Country/Area,18 128,18 467,18 820,19 184,19 560,19 947,20 348,...,90 140,92 727,95 386,98 094,100 835,103 603,106 400,109 224,112 079,114 964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,285,Bermuda,Country/Area,37,38,38,39,40,41,42,...,65,65,64,64,64,63,63,63,63,62
285,286,Canada,Country/Area,13 733,14 078,14 445,14 835,15 245,15 674,16 115,...,34 539,34 922,35 297,35 664,36 027,36 383,36 732,37 075,37 411,37 742
286,287,Greenland,Country/Area,23,23,24,24,25,26,27,...,57,56,56,56,56,56,56,57,57,57
287,288,Saint Pierre and Miquelon,Country/Area,5,5,5,5,5,5,5,...,6,6,6,6,6,6,6,6,6,6


In [30]:
df_pop.columns

Index(['Index', 'Region, subregion, country or area *', 'Type', '1950', '1951',
       '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960',
       '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969',
       '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978',
       '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987',
       '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020'],
      dtype='object')

In [31]:
df_pop=df_pop.drop(columns={'Type','Index'})

In [32]:
df_pop.rename(columns={'Region, subregion, country or area *':'Country'},inplace=True)

In [33]:
df_pop.drop(df_pop.iloc[:,1:52],axis=1,inplace=True)
df_pop

,Country,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
26,Burundi,6 526,6 704,6 909,7 132,7 365,7 608,7 862,8 126,8 398,...,8 958,9 246,9 540,9 844,10 160,10 488,10 827,11 175,11 531,11 891
27,Comoros,556,569,583,597,612,626,642,657,673,...,707,724,742,759,777,796,814,832,851,870
28,Djibouti,733,747,760,772,783,795,805,816,828,...,854,868,883,899,914,929,944,959,974,988
29,Eritrea,2 375,2 481,2 601,2 720,2 827,2 918,2 997,3 063,3 120,...,3 214,3 250,3 281,3 311,3 343,3 377,3 413,3 453,3 497,3 546
30,Ethiopia,68 159,70 142,72 171,74 240,76 346,78 489,80 674,82 916,85 234,...,90 140,92 727,95 386,98 094,100 835,103 603,106 400,109 224,112 079,114 964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,Bermuda,65,66,66,66,66,66,66,66,66,...,65,65,64,64,64,63,63,63,63,62
285,Canada,30 880,31 178,31 488,31 815,32 164,32 537,32 931,33 338,33 746,...,34 539,34 922,35 297,35 664,36 027,36 383,36 732,37 075,37 411,37 742
286,Greenland,56,57,57,57,57,57,57,57,57,...,57,56,56,56,56,56,56,57,57,57
287,Saint Pierre and Miquelon,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6


In [34]:
#lets take a look....

df_pop.reset_index(drop=True)

,Country,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Burundi,6 526,6 704,6 909,7 132,7 365,7 608,7 862,8 126,8 398,...,8 958,9 246,9 540,9 844,10 160,10 488,10 827,11 175,11 531,11 891
1,Comoros,556,569,583,597,612,626,642,657,673,...,707,724,742,759,777,796,814,832,851,870
2,Djibouti,733,747,760,772,783,795,805,816,828,...,854,868,883,899,914,929,944,959,974,988
3,Eritrea,2 375,2 481,2 601,2 720,2 827,2 918,2 997,3 063,3 120,...,3 214,3 250,3 281,3 311,3 343,3 377,3 413,3 453,3 497,3 546
4,Ethiopia,68 159,70 142,72 171,74 240,76 346,78 489,80 674,82 916,85 234,...,90 140,92 727,95 386,98 094,100 835,103 603,106 400,109 224,112 079,114 964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,Bermuda,65,66,66,66,66,66,66,66,66,...,65,65,64,64,64,63,63,63,63,62
231,Canada,30 880,31 178,31 488,31 815,32 164,32 537,32 931,33 338,33 746,...,34 539,34 922,35 297,35 664,36 027,36 383,36 732,37 075,37 411,37 742
232,Greenland,56,57,57,57,57,57,57,57,57,...,57,56,56,56,56,56,56,57,57,57
233,Saint Pierre and Miquelon,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6


In [35]:
#lets add the iso  for merger

iso_countries_df_pop=[]

for x in df_pop['Country']:
    try:
        iso3 = Country.get_iso3_country_code_fuzzy(x)
        info = Country.get_country_info_from_iso3(iso3[0])
        reg=info.get('#country+code+v_iso3')
#        reg = info.get('#country+name+preferred')
        iso_countries_df_pop.append(reg)
        
    except AttributeError:
        iso_countries_df_pop.append('not_country')

df_pop['iso3_codes']=iso_countries_df_pop

In [36]:
#lets make sure we have the same years and that we can produce a clean database

old_columns=df_pop.columns[1:len(df_pop.columns)-1]
old_columns



Index(['2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020'],
      dtype='object')

In [37]:
new_columns=[]

for year in old_columns:
    new_columns.append('C'+year)

new_columns

['C2001',
 'C2002',
 'C2003',
 'C2004',
 'C2005',
 'C2006',
 'C2007',
 'C2008',
 'C2009',
 'C2010',
 'C2011',
 'C2012',
 'C2013',
 'C2014',
 'C2015',
 'C2016',
 'C2017',
 'C2018',
 'C2019',
 'C2020']

In [38]:
new_columns=['Country','C2001',
 'C2002',
 'C2003',
 'C2004',
 'C2005',
 'C2006',
 'C2007',
 'C2008',
 'C2009',
 'C2010',
 'C2011',
 'C2012',
 'C2013',
 'C2014',
 'C2015',
 'C2016',
 'C2017',
 'C2018',
 'C2019',
 'C2020','iso3_codes']

In [39]:
df_pop.columns=new_columns

In [40]:
df_pop

,Country,C2001,C2002,C2003,C2004,C2005,C2006,C2007,C2008,C2009,...,C2012,C2013,C2014,C2015,C2016,C2017,C2018,C2019,C2020,iso3_codes
26,Burundi,6 526,6 704,6 909,7 132,7 365,7 608,7 862,8 126,8 398,...,9 246,9 540,9 844,10 160,10 488,10 827,11 175,11 531,11 891,BDI
27,Comoros,556,569,583,597,612,626,642,657,673,...,724,742,759,777,796,814,832,851,870,COM
28,Djibouti,733,747,760,772,783,795,805,816,828,...,868,883,899,914,929,944,959,974,988,DJI
29,Eritrea,2 375,2 481,2 601,2 720,2 827,2 918,2 997,3 063,3 120,...,3 250,3 281,3 311,3 343,3 377,3 413,3 453,3 497,3 546,ERI
30,Ethiopia,68 159,70 142,72 171,74 240,76 346,78 489,80 674,82 916,85 234,...,92 727,95 386,98 094,100 835,103 603,106 400,109 224,112 079,114 964,ETH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,Bermuda,65,66,66,66,66,66,66,66,66,...,65,64,64,64,63,63,63,63,62,BMU
285,Canada,30 880,31 178,31 488,31 815,32 164,32 537,32 931,33 338,33 746,...,34 922,35 297,35 664,36 027,36 383,36 732,37 075,37 411,37 742,CAN
286,Greenland,56,57,57,57,57,57,57,57,57,...,56,56,56,56,56,56,57,57,57,GRL
287,Saint Pierre and Miquelon,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,SPM


In [41]:
merged_hdi_aidg_pop=merged_hdi_aidg.merge(df_pop, on='iso3_codes')
merged_hdi_aidg_pop

,Country_x,X2001,X2002,X2003,X2004,X2005,X2006,X2007,X2008,X2009,...,C2011,C2012,C2013,C2014,C2015,C2016,C2017,C2018,C2019,C2020
0,Afghanistan,0.347,0.378,0.387,0.400,0.410,0.419,0.431,0.436,0.447,...,30 117,31 161,32 270,33 371,34 414,35 383,36 296,37 172,38 042,38 928
1,Albania,0.673,0.680,0.687,0.692,0.702,0.709,0.718,0.724,0.729,...,2 929,2 914,2 904,2 896,2 891,2 886,2 884,2 883,2 881,2 878
2,Algeria,0.655,0.666,0.676,0.685,0.694,0.699,0.708,0.711,0.720,...,36 661,37 384,38 140,38 924,39 728,40 551,41 389,42 228,43 053,43 851
3,Argentina,0.775,0.770,0.775,0.775,0.777,0.801,0.806,0.810,0.810,...,41 320,41 755,42 196,42 638,43 075,43 508,43 937,44 361,44 781,45 196
4,Armenia,0.653,0.663,0.672,0.681,0.694,0.708,0.724,0.727,0.724,...,2 877,2 884,2 898,2 912,2 926,2 936,2 945,2 952,2 958,2 963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,Venezuela (Bolivarian Republic of),0.680,0.688,0.687,0.700,0.714,0.727,0.743,0.753,0.752,...,28 888,29 361,29 781,30 043,30 082,29 851,29 402,28 887,28 516,28 436
103,Viet Nam,0.586,0.594,0.603,0.612,0.616,0.624,0.632,0.639,0.650,...,88 871,89 802,90 753,91 714,92 677,93 640,94 601,95 546,96 462,97 339
104,Yemen,0.449,0.456,0.464,0.471,0.472,0.478,0.486,0.488,0.503,...,23 808,24 473,25 147,25 823,26 498,27 168,27 835,28 499,29 162,29 826
105,Zambia,0.436,0.445,0.455,0.464,0.475,0.486,0.492,0.507,0.521,...,14 023,14 465,14 927,15 400,15 879,16 363,16 854,17 352,17 861,18 384


In [42]:
merged_hdi_aidg_pop.columns

Index(['Country_x', 'X2001', 'X2002', 'X2003', 'X2004', 'X2005', 'X2006',
       'X2007', 'X2008', 'X2009', 'X2010', 'X2011', 'X2012', 'X2013', 'X2014',
       'X2015', 'X2016', 'X2017', 'X2018', 'iso3_codes', 'Y2001', 'Y2002',
       'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009', 'Y2010',
       'Y2011', 'Y2012', 'Y2013', 'Y2014', 'Y2015', 'Y2016', 'Y2017', 'Y2018',
       'Country_y', 'C2001', 'C2002', 'C2003', 'C2004', 'C2005', 'C2006',
       'C2007', 'C2008', 'C2009', 'C2010', 'C2011', 'C2012', 'C2013', 'C2014',
       'C2015', 'C2016', 'C2017', 'C2018', 'C2019', 'C2020'],
      dtype='object')

In [43]:
merged_hdi_aidg_pop['id']=merged_hdi_aidg_pop.index
merged_hdi_aidg_pop

,Country_x,X2001,X2002,X2003,X2004,X2005,X2006,X2007,X2008,X2009,...,C2012,C2013,C2014,C2015,C2016,C2017,C2018,C2019,C2020,id
0,Afghanistan,0.347,0.378,0.387,0.400,0.410,0.419,0.431,0.436,0.447,...,31 161,32 270,33 371,34 414,35 383,36 296,37 172,38 042,38 928,0
1,Albania,0.673,0.680,0.687,0.692,0.702,0.709,0.718,0.724,0.729,...,2 914,2 904,2 896,2 891,2 886,2 884,2 883,2 881,2 878,1
2,Algeria,0.655,0.666,0.676,0.685,0.694,0.699,0.708,0.711,0.720,...,37 384,38 140,38 924,39 728,40 551,41 389,42 228,43 053,43 851,2
3,Argentina,0.775,0.770,0.775,0.775,0.777,0.801,0.806,0.810,0.810,...,41 755,42 196,42 638,43 075,43 508,43 937,44 361,44 781,45 196,3
4,Armenia,0.653,0.663,0.672,0.681,0.694,0.708,0.724,0.727,0.724,...,2 884,2 898,2 912,2 926,2 936,2 945,2 952,2 958,2 963,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,Venezuela (Bolivarian Republic of),0.680,0.688,0.687,0.700,0.714,0.727,0.743,0.753,0.752,...,29 361,29 781,30 043,30 082,29 851,29 402,28 887,28 516,28 436,102
103,Viet Nam,0.586,0.594,0.603,0.612,0.616,0.624,0.632,0.639,0.650,...,89 802,90 753,91 714,92 677,93 640,94 601,95 546,96 462,97 339,103
104,Yemen,0.449,0.456,0.464,0.471,0.472,0.478,0.486,0.488,0.503,...,24 473,25 147,25 823,26 498,27 168,27 835,28 499,29 162,29 826,104
105,Zambia,0.436,0.445,0.455,0.464,0.475,0.486,0.492,0.507,0.521,...,14 465,14 927,15 400,15 879,16 363,16 854,17 352,17 861,18 384,105


In [44]:

final_merged_hdi_aidg_pop=pd.wide_to_long(merged_hdi_aidg_pop, ['X','Y','C'],i='id',j='year')

final_merged_hdi_aidg_pop.reset_index(drop=False, inplace=True)



In [45]:
#this is a clean database

final_merged_hdi_aidg_pop

,id,year,iso3_codes,Country_y,Country_x,X,Y,C
0,0,2001,AFG,Afghanistan,Afghanistan,0.347,5.745670e+07,21 607
1,0,2002,AFG,Afghanistan,Afghanistan,0.378,5.549478e+08,22 601
2,0,2003,AFG,Afghanistan,Afghanistan,0.387,9.576053e+08,23 681
3,0,2004,AFG,Afghanistan,Afghanistan,0.400,1.648764e+09,24 727
4,0,2005,AFG,Afghanistan,Afghanistan,0.410,2.143609e+09,25 654
...,...,...,...,...,...,...,...,...
2135,106,2016,ZWE,Zimbabwe,Zimbabwe,0.549,2.363469e+08,14 030
2136,106,2017,ZWE,Zimbabwe,Zimbabwe,0.553,2.362431e+08,14 237
2137,106,2018,ZWE,Zimbabwe,Zimbabwe,0.563,2.312752e+08,14 439
2138,106,2019,ZWE,Zimbabwe,Zimbabwe,NaN,NaN,14 645


In [46]:
#lets drop what we don't need...

final_merged_hdi_aidg_pop.drop(['id','Country_y'],axis=1,inplace=True)


In [47]:
#rename columns...

final_merged_hdi_aidg_pop.rename(columns={'X':'hdi','Y':'aid_gross','C':'population','Country_x':'country','iso3_codes':
                                        'iso'},inplace=True)

In [48]:
#one more df we want to look at is US aid per sector, managed by USAID.
df_sector_aid=pd.read_csv(r'us_aid.csv')

C:\Users\dccam\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (43) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [49]:
df_sector_aid.head()

,country_id,country_code,country_name,region_id,region_name,income_group_id,income_group_name,income_group_acronym,implementing_agency_id,implementing_agency_acronym,...,activity_start_date,activity_end_date,transaction_type_id,transaction_type_name,fiscal_year,current_amount,constant_amount,USG_sector_id,USG_sector_name,submission_id
0,4,AFG,Afghanistan,4,South and Central Asia,1.0,Low Income Country,LIC,7,DOD,...,NaN,NaN,2,Obligations,2011,9941000000,11391531078,3,Stabilization Operations and Security Sector R...,28
1,4,AFG,Afghanistan,4,South and Central Asia,1.0,Low Income Country,LIC,7,DOD,...,NaN,NaN,2,Obligations,2012,9243000000,10394677800,3,Stabilization Operations and Security Sector R...,28
2,4,AFG,Afghanistan,4,South and Central Asia,1.0,Low Income Country,LIC,7,DOD,...,NaN,NaN,3,Disbursements,2011,7840175215,8984166544,3,Stabilization Operations and Security Sector R...,28
3,4,AFG,Afghanistan,4,South and Central Asia,1.0,Low Income Country,LIC,7,DOD,...,NaN,NaN,3,Disbursements,2013,7764310985,8573982850,3,Stabilization Operations and Security Sector R...,28
4,4,AFG,Afghanistan,4,South and Central Asia,1.0,Low Income Country,LIC,7,DOD,...,NaN,NaN,2,Obligations,2013,6928000000,7650460330,3,Stabilization Operations and Security Sector R...,28


In [50]:
df_sector_aid[df_sector_aid['implementing_agency_acronym']=='USAID']['USG_sector_name'].value_counts()

#filter for USAID, Disbursements

df_sector_aid=df_sector_aid[(df_sector_aid['implementing_agency_acronym']=='USAID') & (df_sector_aid['transaction_type_name']=='Disbursements')]

In [51]:
#let's grab the info we are really interested in

df_sector_aid=df_sector_aid[['income_group_name','country_code', 'country_name','fiscal_year','constant_amount','USG_sector_id','USG_sector_name']]
df_sector_aid.reset_index(drop=True,inplace=True)


In [52]:
df_sector_aid['USG_sector_name']=df_sector_aid['USG_sector_name'].str.strip()

In [53]:
#lets drop some info within the columns which we did want (World, general administrative expenses...)

indexNames = df_sector_aid[(df_sector_aid['country_code'] == 'WLD') | (df_sector_aid['USG_sector_name'].isin(['Direct Administrative Costs','Monitoring and Evaluation','Multi-sector - Unspecified']))].index
df_sector_aid.drop(indexNames, inplace=True)

#make sure they are infact gone...
df_sector_aid['USG_sector_name'].value_counts().sort_index()

Agriculture                                              14451
Basic Education                                          11525
Civil Society                                            11194
Clean Productive Environment                              7999
Combating Weapons of Mass Destruction (WMD)                 20
Conflict Mitigation and Reconciliation                    8392
Counter-Narcotics                                         1082
Counter-Terrorism                                          651
Disaster Readiness                                        3436
Economic Opportunity                                      3349
Environment                                                 14
Family Planning and Reproductive Health                   9484
Financial Sector                                          2090
Good Governance                                          21069
HIV/AIDS                                                 30122
Health - General                                       

In [54]:
df_sector_aid.reset_index(drop=True, inplace=True)

In [55]:
#lets take a look....
#its already in the right format...
#we cant merge this into the other dataframe because of sectors

df_sector_aid.drop(['USG_sector_id'],axis=1,inplace=True)
df_sector_aid.rename(columns={'USG_sector_name':'sector','country_name':'country','fiscal_year':'year'},inplace=True)
df_sector_aid

,income_group_name,country_code,country,year,constant_amount,sector
0,High Income Country,ISR,Israel,2001,1180593361,Good Governance
1,Upper Middle Income Country,IRQ,Iraq,2005,994439877,Infrastructure
2,Upper Middle Income Country,JOR,Jordan,2019,745100000,Macroeconomic Foundation for Growth
3,Upper Middle Income Country,JOR,Jordan,2020,730676195,Macroeconomic Foundation for Growth
4,Upper Middle Income Country,JOR,Jordan,2003,953050847,"Protection, Assistance and Solutions"
...,...,...,...,...,...,...
270614,Low Income Country,SLE,Sierra Leone,2017,-15646972,Maternal and Child Health
270615,Lower Middle Income Country,PAK,Pakistan,2015,-20947102,Infrastructure
270616,Upper Middle Income Country,RUS,Russia,2005,-29113294,Good Governance
270617,Lower Middle Income Country,PSE,West Bank/Gaza,2006,-56261256,Good Governance


In [56]:
#rename some columns

df_sector_aid.rename(columns={'country_code':'iso'},inplace=True)
df_sector_aid

,income_group_name,iso,country,year,constant_amount,sector
0,High Income Country,ISR,Israel,2001,1180593361,Good Governance
1,Upper Middle Income Country,IRQ,Iraq,2005,994439877,Infrastructure
2,Upper Middle Income Country,JOR,Jordan,2019,745100000,Macroeconomic Foundation for Growth
3,Upper Middle Income Country,JOR,Jordan,2020,730676195,Macroeconomic Foundation for Growth
4,Upper Middle Income Country,JOR,Jordan,2003,953050847,"Protection, Assistance and Solutions"
...,...,...,...,...,...,...
270614,Low Income Country,SLE,Sierra Leone,2017,-15646972,Maternal and Child Health
270615,Lower Middle Income Country,PAK,Pakistan,2015,-20947102,Infrastructure
270616,Upper Middle Income Country,RUS,Russia,2005,-29113294,Good Governance
270617,Lower Middle Income Country,PSE,West Bank/Gaza,2006,-56261256,Good Governance


In [57]:
#lets add hdi per year to the sector_aid df so we can run regression between the two variables

df_sector_aid=pd.merge(df_sector_aid, final_merged_hdi_aidg_pop, on='iso',how='right')

df_sector_aid

,income_group_name,iso,country_x,year_x,constant_amount,sector,year_y,country_y,hdi,aid_gross,population
0,High Income Country,ISR,Israel,2001,1.180593e+09,Good Governance,2001,Israel,0.857,4.093436e+09,6 061
1,High Income Country,ISR,Israel,2003,8.115909e+08,Financial Sector,2001,Israel,0.857,4.093436e+09,6 061
2,High Income Country,ISR,Israel,2002,7.903296e+08,Financial Sector,2001,Israel,0.857,4.093436e+09,6 061
3,High Income Country,ISR,Israel,2004,6.344563e+08,Financial Sector,2001,Israel,0.857,4.093436e+09,6 061
4,High Income Country,ISR,Israel,2005,4.607815e+08,Financial Sector,2001,Israel,0.857,4.093436e+09,6 061
...,...,...,...,...,...,...,...,...,...,...,...
3656435,NaN,ARG,NaN,NaN,NaN,NaN,2016,Argentina,0.828,6.539909e+06,43 508
3656436,NaN,ARG,NaN,NaN,NaN,NaN,2017,Argentina,0.832,2.689528e+06,43 937
3656437,NaN,ARG,NaN,NaN,NaN,NaN,2018,Argentina,0.830,5.546173e+06,44 361
3656438,NaN,ARG,NaN,NaN,NaN,NaN,2019,Argentina,NaN,NaN,44 781


In [58]:
#rename some columns...

df_sector_aid.drop(['country_y','year_y','aid_gross'],inplace=True,axis=1)

df_sector_aid

,income_group_name,iso,country_x,year_x,constant_amount,sector,hdi,population
0,High Income Country,ISR,Israel,2001,1.180593e+09,Good Governance,0.857,6 061
1,High Income Country,ISR,Israel,2003,8.115909e+08,Financial Sector,0.857,6 061
2,High Income Country,ISR,Israel,2002,7.903296e+08,Financial Sector,0.857,6 061
3,High Income Country,ISR,Israel,2004,6.344563e+08,Financial Sector,0.857,6 061
4,High Income Country,ISR,Israel,2005,4.607815e+08,Financial Sector,0.857,6 061
...,...,...,...,...,...,...,...,...
3656435,NaN,ARG,NaN,NaN,NaN,NaN,0.828,43 508
3656436,NaN,ARG,NaN,NaN,NaN,NaN,0.832,43 937
3656437,NaN,ARG,NaN,NaN,NaN,NaN,0.830,44 361
3656438,NaN,ARG,NaN,NaN,NaN,NaN,NaN,44 781


In [59]:
df_sector_aid.rename(columns={'country_x':'country','year_x':'year','constant_amount':'aid_sector_amount'},inplace=True)

df_sector_aid

,income_group_name,iso,country,year,aid_sector_amount,sector,hdi,population
0,High Income Country,ISR,Israel,2001,1.180593e+09,Good Governance,0.857,6 061
1,High Income Country,ISR,Israel,2003,8.115909e+08,Financial Sector,0.857,6 061
2,High Income Country,ISR,Israel,2002,7.903296e+08,Financial Sector,0.857,6 061
3,High Income Country,ISR,Israel,2004,6.344563e+08,Financial Sector,0.857,6 061
4,High Income Country,ISR,Israel,2005,4.607815e+08,Financial Sector,0.857,6 061
...,...,...,...,...,...,...,...,...
3656435,NaN,ARG,NaN,NaN,NaN,NaN,0.828,43 508
3656436,NaN,ARG,NaN,NaN,NaN,NaN,0.832,43 937
3656437,NaN,ARG,NaN,NaN,NaN,NaN,0.830,44 361
3656438,NaN,ARG,NaN,NaN,NaN,NaN,NaN,44 781


In [60]:
#lets get rid of null values from both dfs....

df_sector_aid.isnull().sum()

income_group_name        20
iso                       0
country                  20
year                     20
aid_sector_amount        20
sector                   20
hdi                  365644
population                0
dtype: int64

In [61]:
df_sector_aid.shape

(3656440, 8)

In [62]:
df_sector_aid.dropna(axis=0,inplace=True)

In [63]:
df_sector_aid.shape

(3290778, 8)

In [64]:
final_merged_hdi_aidg_pop.isnull().sum()

year            0
iso             0
country         0
hdi           214
aid_gross     214
population      0
dtype: int64

In [65]:
final_merged_hdi_aidg_pop.shape

(2140, 6)

In [66]:
final_merged_hdi_aidg_pop.dropna(axis=0,inplace=True)

In [67]:
final_merged_hdi_aidg_pop.shape

(1926, 6)

Lets explore the data...

In [68]:
#lets make sure we have avery data point as the right element type

final_merged_hdi_aidg_pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1926 entries, 0 to 2137
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        1926 non-null   int64  
 1   iso         1926 non-null   object 
 2   country     1926 non-null   object 
 3   hdi         1926 non-null   object 
 4   aid_gross   1926 non-null   float64
 5   population  1926 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 105.3+ KB


In [69]:
#lets make change things so that  we have avery data point as the right element type

#year -->nominal
#iso --> nominal 
#country --> nominal

#hdi -> conitnuous
#aid_gross --> continuous
#population --> contiuous


final_merged_hdi_aidg_pop['population']=final_merged_hdi_aidg_pop['population'].str.replace(' ','')
final_merged_hdi_aidg_pop.population=final_merged_hdi_aidg_pop.population.astype(int)

final_merged_hdi_aidg_pop.hdi=final_merged_hdi_aidg_pop.hdi.astype(float)

final_merged_hdi_aidg_pop.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 1926 entries, 0 to 2137
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        1926 non-null   int64  
 1   iso         1926 non-null   object 
 2   country     1926 non-null   object 
 3   hdi         1926 non-null   float64
 4   aid_gross   1926 non-null   float64
 5   population  1926 non-null   int32  
dtypes: float64(2), int32(1), int64(1), object(2)
memory usage: 97.8+ KB


In [70]:
#let match millions in population to that of aid gross & calculate aid per capita(in millions)

final_merged_hdi_aidg_pop['aid_gross']=final_merged_hdi_aidg_pop['aid_gross']/1000000
final_merged_hdi_aidg_pop['aid_per_cap']=final_merged_hdi_aidg_pop['aid_gross']/final_merged_hdi_aidg_pop['population']
final_merged_hdi_aidg_pop.head()

,year,iso,country,hdi,aid_gross,population,aid_per_cap
0,2001,AFG,Afghanistan,0.347,57.456696,21607,0.002659
1,2002,AFG,Afghanistan,0.378,554.947819,22601,0.024554
2,2003,AFG,Afghanistan,0.387,957.605301,23681,0.040438
3,2004,AFG,Afghanistan,0.400,1648.763748,24727,0.066679
4,2005,AFG,Afghanistan,0.410,2143.609027,25654,0.083558


In [71]:
final_merged_hdi_aidg_pop.head()

,year,iso,country,hdi,aid_gross,population,aid_per_cap
0,2001,AFG,Afghanistan,0.347,57.456696,21607,0.002659
1,2002,AFG,Afghanistan,0.378,554.947819,22601,0.024554
2,2003,AFG,Afghanistan,0.387,957.605301,23681,0.040438
3,2004,AFG,Afghanistan,0.400,1648.763748,24727,0.066679
4,2005,AFG,Afghanistan,0.410,2143.609027,25654,0.083558


In [72]:
final_merged_hdi_aidg_pop['intercept']=1

lm=sm.OLS(final_merged_hdi_aidg_pop['hdi'],final_merged_hdi_aidg_pop['aid_per_cap'])

In [73]:
results=lm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    hdi   R-squared (uncentered):                   0.090
Model:                            OLS   Adj. R-squared (uncentered):              0.090
Method:                 Least Squares   F-statistic:                              191.4
Date:                Sat, 02 Jan 2021   Prob (F-statistic):                    1.45e-41
Time:                        20:51:44   Log-Likelihood:                         -1820.1
No. Observations:                1926   AIC:                                      3642.
Df Residuals:                    1925   BIC:                                      3648.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
aid_per_cap     3.2020      0.231     13.834      0.000       2.748       3.656
==============================================================================
Omnibus:                     1315.123   Durbin-Watson:                   0.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            26180.585
Skew:                          -2.939   Prob(JB):                         0.00
Kurtosis:                      20.079   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [74]:
#lets make sure we have avery data point as the right element type for sector, USAID df.

df_sector_aid['population']=df_sector_aid['population'].str.replace(' ','')
df_sector_aid.population=df_sector_aid.population.astype(int)
print(df_sector_aid['population'])

0          6061
1          6061
2          6061
3          6061
4          6061
           ... 
3656411    1317
3656412    1319
3656413    1319
3656414    1323
3656415    1323
Name: population, Length: 3290778, dtype: int32


In [75]:
df_sector_aid.hdi=df_sector_aid.hdi.astype(float)
df_sector_aid.info()

#these are in the correct element type:

#iso -->  nominal variable
#country --> nominal variable
#year --> nominal variable

#aid_sector_amount--> continuoues variable
#sector --> categorical variable
#population --> continous variable
#hdi --> continuous variable



<class 'pandas.core.frame.DataFrame'>
Int64Index: 3290778 entries, 0 to 3656415
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   income_group_name  object 
 1   iso                object 
 2   country            object 
 3   year               object 
 4   aid_sector_amount  float64
 5   sector             object 
 6   hdi                float64
 7   population         int32  
dtypes: float64(2), int32(1), object(5)
memory usage: 213.4+ MB


In [76]:
df_sector_aid['aid_sector_amount'][0]

1180593361.0

In [77]:
#let match millions in population to that of aid gross & calculate aid per capita(in millions)

df_sector_aid['aid_sector_amount']=df_sector_aid['aid_sector_amount']/1000000
df_sector_aid['aid_sector_amount_per_cap']=df_sector_aid['aid_sector_amount']/df_sector_aid['population']
df_sector_aid.head()


,income_group_name,iso,country,year,aid_sector_amount,sector,hdi,population,aid_sector_amount_per_cap
0,High Income Country,ISR,Israel,2001,1180.593361,Good Governance,0.857,6061,0.194785
1,High Income Country,ISR,Israel,2003,811.590872,Financial Sector,0.857,6061,0.133904
2,High Income Country,ISR,Israel,2002,790.329563,Financial Sector,0.857,6061,0.130396
3,High Income Country,ISR,Israel,2004,634.456293,Financial Sector,0.857,6061,0.104678
4,High Income Country,ISR,Israel,2005,460.781496,Financial Sector,0.857,6061,0.076024


In [78]:
df_dummies_sector_aid = pd.get_dummies(df_sector_aid, columns=['sector'])  
print(df_dummies_sector_aid.columns)


Index(['income_group_name', 'iso', 'country', 'year', 'aid_sector_amount',
       'hdi', 'population', 'aid_sector_amount_per_cap', 'sector_Agriculture',
       'sector_Basic Education', 'sector_Civil Society',
       'sector_Clean Productive Environment',
       'sector_Combating Weapons of Mass Destruction (WMD)',
       'sector_Conflict Mitigation and Reconciliation',
       'sector_Counter-Narcotics', 'sector_Counter-Terrorism',
       'sector_Disaster Readiness', 'sector_Economic Opportunity',
       'sector_Environment', 'sector_Family Planning and Reproductive Health',
       'sector_Financial Sector', 'sector_Good Governance', 'sector_HIV/AIDS',
       'sector_Health - General', 'sector_Higher Education',
       'sector_Infrastructure', 'sector_Macroeconomic Foundation for Growth',
       'sector_Malaria', 'sector_Maternal and Child Health',
       'sector_Natural Resources and Biodiversity', 'sector_Nutrition',
       'sector_Other Public Health Threats',
       'sector_Pandem

In [79]:
df_dummies_sector_aid.drop(df_dummies_sector_aid.columns[8],axis=1,inplace=True)

In [80]:

df_dummies_sector_aid['intercept']=1

lm=sm.OLS(df_dummies_sector_aid['hdi'],df_dummies_sector_aid[['intercept',
       'sector_Basic Education', 'sector_Civil Society',
       'sector_Clean Productive Environment',
       'sector_Combating Weapons of Mass Destruction (WMD)',
       'sector_Conflict Mitigation and Reconciliation',
       'sector_Counter-Narcotics', 'sector_Counter-Terrorism',
       'sector_Disaster Readiness', 'sector_Economic Opportunity',
       'sector_Environment', 'sector_Family Planning and Reproductive Health',
       'sector_Financial Sector', 'sector_Good Governance', 'sector_HIV/AIDS',
       'sector_Health - General', 'sector_Higher Education',
       'sector_Infrastructure', 'sector_Macroeconomic Foundation for Growth',
       'sector_Malaria', 'sector_Maternal and Child Health',
       'sector_Natural Resources and Biodiversity', 'sector_Nutrition',
       'sector_Other Public Health Threats',
       'sector_Pandemic Influenza and Other Emerging Threats (PIOET)',
       'sector_Policies, Regulations, and Systems',
       'sector_Political Competition and Consensus-Building',
       'sector_Private Sector Competitiveness',
       'sector_Protection, Assistance and Solutions',
       'sector_Rule of Law and Human Rights', 'sector_Social Assistance',
       'sector_Social Services',
       'sector_Stabilization Operations and Security Sector Reform',
       'sector_Trade and Investment', 'sector_Transnational Crime',
       'sector_Tuberculosis', 'sector_Water Supply and Sanitation']])

In [81]:
results=lm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    hdi   R-squared:                       0.118
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                 1.220e+04
Date:                Sat, 02 Jan 2021   Prob (F-statistic):               0.00
Time:                        20:52:13   Log-Likelihood:             2.4152e+06
No. Observations:             3290778   AIC:                        -4.830e+06
Df Residuals:                 3290741   BIC:                        -4.830e+06
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
================================================================================================================================
                                                                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------------
intercept                                                        0.5198      0.000   1874.498      0.000       0.519       0.520
sector_Basic Education                                           0.0265      0.000     65.669      0.000       0.026       0.027
sector_Civil Society                                             0.1103      0.000    260.071      0.000       0.109       0.111
sector_Clean Productive Environment                              0.1051      0.000    224.249      0.000       0.104       0.106
sector_Combating Weapons of Mass Destruction (WMD)               0.2073      0.007     31.200      0.000       0.194       0.220
sector_Conflict Mitigation and Reconciliation                    0.0344      0.000     70.827      0.000       0.033       0.035
sector_Counter-Narcotics                                         0.1250      0.001    123.545      0.000       0.123       0.127
sector_Counter-Terrorism                                         0.0164      0.002      9.718      0.000       0.013       0.020
sector_Disaster Readiness                                        0.0585      0.001     87.996      0.000       0.057       0.060
sector_Economic Opportunity                                      0.0898      0.001    141.878      0.000       0.089       0.091
sector_Environment                                               0.0534      0.019      2.757      0.006       0.015       0.091
sector_Family Planning and Reproductive Health                   0.0234      0.000     54.731      0.000       0.023       0.024
sector_Financial Sector                                          0.1213      0.001    152.902      0.000       0.120       0.123
sector_Good Governance                                           0.0720      0.000    203.232      0.000       0.071       0.073
sector_HIV/AIDS                                                  0.0157      0.000     47.350      0.000       0.015       0.016
sector_Health - General                                         -0.0474      0.007     -6.700      0.000      -0.061      -0.034
sector_Higher Education                                          0.0996      0.001    127.843      0.000       0.098       0.101
sector_Infrastructure                                            0.0559      0.001    101.282      0.000       0.055       0.057
sector_Macroeconomic Foundation for Growth                       0.1195      0.001    114.872      0.000       0.117       0.122
sector_Malaria                                                  -0.0524      0.001    -97.581      0.000      -0.053      -0.051
sector_Maternal and Child Health                                 0.0107      0.000     25.443      0.000       0.010

In [82]:
#lets create a for loop to run regressions per  income group

for sector in df_sector_aid['income_group_name'].unique():
    sector_specific_df=df_sector_aid[df_sector_aid['income_group_name']==sector]
    sector_specific_df= pd.get_dummies(sector_specific_df, columns=['sector'])  
    sector_specific_df['intercept']=1
    columns=sector_specific_df.columns[9:]
    
    lm=sm.OLS(sector_specific_df['hdi'],sector_specific_df[columns])
    results=lm.fit()
    
    print("- - - - ",sector,"- - - - ")
    print(" ")
    print(" ")
    print(" ")
    print("- - - - ",sector_specific_df['country'].unique(),"- - - - ")
    print(" ")
    print(" ")
    print(" ")
    print(results.summary())
    print("- - - - ",'done',"- - - - ")
    print(" ")
    print(" ")
    print(" ")
    print(" ")

- - - -  High Income Country - - - - 
 
 
 
- - - -  ['Israel' 'Cyprus' 'Chile' 'Poland' 'Panama' 'Japan' 'Hungary' 'Greece'
 'Lithuania' 'Czechia' 'Barbados' 'Trinidad and Tobago' 'Latvia'
 'Portugal' 'Canada' 'Bahrain' 'Estonia'] - - - - 
 
 
 
                            OLS Regression Results                            
Dep. Variable:                    hdi   R-squared:                       0.522
Model:                            OLS   Adj. R-squared:                  0.521
Method:                 Least Squares   F-statistic:                     979.6
Date:                Sat, 02 Jan 2021   Prob (F-statistic):               0.00
Time:                        20:52:15   Log-Likelihood:                 43189.
No. Observations:               22464   AIC:                        -8.633e+04
Df Residuals:                   22438   BIC:                        -8.612e+04
Df Model:                          25                                         
Covariance Type:            nonrobust     

- - - -  Lower Middle Income Country - - - - 
 
 
 
- - - -  ['Pakistan' 'Egypt' 'Sudan' 'Kenya' 'Tunisia' 'Indonesia' 'Bangladesh'
 'Zambia' 'Ukraine' "Cote d'Ivoire" 'India' 'Bolivia' 'Philippines'
 'Honduras' 'Cameroon' 'Sri Lanka' 'Lesotho' 'El Salvador' 'Ghana'
 'Burma (Myanmar)' 'Vietnam' 'Eswatini' 'Kyrgyzstan' 'Mongolia' 'Cambodia'
 'Nicaragua' 'Moldova' 'Morocco' 'Mauritania' 'Papua New Guinea'
 'Congo (Brazzaville)' 'Sao Tome and Principe'] - - - - 
 
 
 
                            OLS Regression Results                            
Dep. Variable:                    hdi   R-squared:                       0.116
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     4739.
Date:                Sat, 02 Jan 2021   Prob (F-statistic):               0.00
Time:                        20:52:24   Log-Likelihood:             1.4838e+06
No. Observations:             1298178   AIC:            